## 加载预训练模型

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 加载预训练的 GPT-2 模型和分词器
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

print(model)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


## 应用 LoRA


In [3]:
from peft import get_peft_model, LoraConfig, TaskType

# 配置 LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 任务类型：因果语言模型
    inference_mode=False,          # 推理模式关闭，以进行训练
    r=8,                           # 低秩值 r
    lora_alpha=32,                 # LoRA 的缩放因子
    lora_dropout=0.1,              # Dropout 概率
)

# 将 LoRA 应用到模型中
model = get_peft_model(model, lora_config)
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): GPT2LMHeadModel(
          (transformer): GPT2Model(
            (wte): Embedding(50257, 768)
            (wpe): Embedding(1024, 768)
            (drop): Dropout(p=0.1, inplace=False)
            (h): ModuleList(
              (0-11): 12 x GPT2Block(
                (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                (attn): GPT2SdpaAttention(
                  (c_attn): lora.Linear(
                    (base_layer): Conv1D()
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2304, bias=False)
  

In [4]:
# 查看 LoRA 模块
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


In [6]:
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

# 1. 使用英文小说数据集
# dataset = load_dataset("roneneldan/TinyStories", split="train[:1000]")  # 取前1000条

# 2. 使用 imdb 电影评论数据集
dataset = load_dataset("imdb", split="train[:500]")

print(f"数据集大小: {len(dataset)}")
print(f"数据集列名: {dataset.column_names}")
print(f"示例数据: {dataset[0]}")

def preprocess_function(examples):
    """数据预处理函数"""
    # 对于IMDB数据集，我们使用 'text' 字段
    # 你可以根据不同数据集调整字段名
    texts = examples['text']
    
    # Tokenization
    model_inputs = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=512,  # 根据需要调整最大长度
        return_tensors="pt" if len(texts) == 1 else None
    )
    
    # 对于因果语言模型，labels就是input_ids
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    
    return model_inputs

# 应用预处理
print("正在预处理数据...")
train_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names,  # 移除原始列，只保留模型需要的
    desc="Tokenizing dataset"
)

print(f"预处理后的数据集大小: {len(train_dataset)}")
print(f"预处理后的数据集特征: {train_dataset.features}")

# 创建数据整理器
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # 不使用masked language modeling，因为我们做的是causal LM
    pad_to_multiple_of=8  # 为了提高效率，填充到8的倍数
)

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

KeyboardInterrupt: 

## 开始微调

In [ ]:
from transformers import Trainer, TrainingArguments

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',         # 模型保存和日志输出的目录路径
    num_train_epochs=3,             # 训练的总轮数（epochs）
    per_device_train_batch_size=4,  # 每个设备（如GPU或CPU）上的训练批次大小，4表示每次输入模型的数据数量
    learning_rate=5e-5,             # 学习率
    logging_steps=10,               # 每隔多少步（steps）进行一次日志记录
    save_steps=100,                 # 每隔多少步保存模型
)

# 创建 Trainer
trainer = Trainer(
    model=model,                    # 训练的模型对象，需要事先加载好
    args=training_args,             # 上面定义的训练参数配置
    train_dataset=train_dataset,    # 使用预处理后的数据集
    data_collator=data_collator     # 数据整理器
)

# 开始训练
trainer.train()

## 保存和加载 LoRA 微调的模型

In [ ]:
# 保存 LoRA 参数
model.save_pretrained('./lora_model')

## 预训练模型 vs lora参数

In [ ]:
# 加载原始模型
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

# 加载 LoRA 参数
from peft import PeftModel

model = PeftModel.from_pretrained(base_model, './lora_model')

## 合并 LoRA 权重并卸载 PEFT 包装

In [ ]:
# merge_and_unload() 将 LoRA 的权重合并回原始模型
# 对比合并前后的模型
print("合并前的模型结构：")
print(model)

# 合并并卸载 LoRA 权重
model = model.merge_and_unload()

print("合并后的模型结构：")
print(model)

合并后模型的 LoRA 层将被去除。lora参数已经合并到正常模型中，此时合并后的模型已经被lora进行微调

In [ ]:
# 保存合并后的模型
model.save_pretrained('./merged_model')
tokenizer.save_pretrained('./merged_model')

在推理阶段，直接加载这个合并后的模型：

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 加载合并后的模型
tokenizer = AutoTokenizer.from_pretrained('./merged_model')
model = AutoModelForCausalLM.from_pretrained('./merged_model')

# 进行推理
inputs = tokenizer("Hello, World！", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))